In [115]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split

In [116]:
df_char=pd.read_csv('training_images/training_images.csv')
df_digits=pd.read_csv('training_images/training_images_digit.csv')

In [117]:
df_final=pd.concat([df_char,df_digits],axis=0)

In [118]:
df_final

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_01_ka
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_01_ka
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_01_ka
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_01_ka
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,character_01_ka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9
19996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,digit_9


In [119]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df_final['label']=le.fit_transform(df_final['label'])


In [120]:
df_final['label'].value_counts()

label
0     2000
34    2000
25    2000
26    2000
27    2000
28    2000
29    2000
30    2000
31    2000
32    2000
33    2000
35    2000
1     2000
36    2000
37    2000
38    2000
39    2000
40    2000
41    2000
42    2000
43    2000
44    2000
24    2000
23    2000
22    2000
21    2000
2     2000
3     2000
4     2000
5     2000
6     2000
7     2000
8     2000
9     2000
10    2000
11    2000
12    2000
13    2000
14    2000
15    2000
16    2000
17    2000
18    2000
19    2000
20    2000
45    2000
Name: count, dtype: int64

In [121]:
X=df_final.drop('label',axis=1)
y=df_final['label']


In [122]:
model=Sequential()
model.add(Dense(512, input_dim=1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(46, activation='softmax'))


In [123]:
from tensorflow.keras.callbacks import EarlyStopping
callback=EarlyStopping(monitor='val_loss',
                       patience=20,
                       restore_best_weights=False,
                       verbose=1,
                       mode='auto',
                    min_delta=0.0001)


In [124]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [126]:
model.fit(X_train, y_train, epochs=20,batch_size=128,callbacks=callback,validation_split=0.2)

Epoch 1/20
460/460 [==============================] - 9s 12ms/step - loss: 1.2753 - accuracy: 0.6510 - val_loss: 0.6180 - val_accuracy: 0.8195
Epoch 2/20
460/460 [==============================] - 6s 13ms/step - loss: 0.4502 - accuracy: 0.8673 - val_loss: 0.4250 - val_accuracy: 0.8735
Epoch 3/20
460/460 [==============================] - 8s 17ms/step - loss: 0.2804 - accuracy: 0.9141 - val_loss: 0.3546 - val_accuracy: 0.8969
Epoch 4/20
460/460 [==============================] - 4s 9ms/step - loss: 0.2013 - accuracy: 0.9371 - val_loss: 0.3155 - val_accuracy: 0.9094
Epoch 5/20
460/460 [==============================] - 4s 8ms/step - loss: 0.1540 - accuracy: 0.9514 - val_loss: 0.3083 - val_accuracy: 0.9103
Epoch 6/20
460/460 [==============================] - 4s 9ms/step - loss: 0.1274 - accuracy: 0.9584 - val_loss: 0.3361 - val_accuracy: 0.9088
Epoch 7/20
460/460 [==============================] - 5s 10ms/step - loss: 0.1016 - accuracy: 0.9672 - val_loss: 0.3092 - val_accuracy: 0.9198
Ep

In [127]:
import pickle

# Serialize the model to a pkl file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://2d72a2d0-4ac3-4e07-a955-1c02f7061a8e/assets


In [128]:
model.predict(X_test)

575/575 [==============================] - 2s 2ms/step


array([[2.0122675e-27, 6.5316249e-22, 1.0886639e-19, ..., 4.1601601e-20,
        2.7615960e-21, 7.0603840e-20],
       [1.6461517e-08, 1.7104181e-08, 5.4740218e-10, ..., 3.3787095e-11,
        1.8835230e-17, 1.8304489e-08],
       [1.7693208e-03, 3.8103310e-05, 1.4896366e-08, ..., 1.7764842e-07,
        4.4503388e-08, 1.4178356e-05],
       ...,
       [2.4016439e-27, 5.5921537e-27, 1.1013945e-29, ..., 1.0578599e-22,
        2.5078908e-28, 1.1166849e-27],
       [3.3134655e-09, 1.2591106e-14, 3.0138329e-13, ..., 3.7640556e-13,
        5.1819697e-14, 8.0510672e-16],
       [1.5832342e-08, 2.5615664e-05, 1.1983075e-13, ..., 6.0108494e-20,
        5.2183764e-19, 3.5021177e-18]], dtype=float32)

In [129]:
from PIL import Image
img=Image.open('ka_1.png').convert('L')

In [130]:
img_array=np.array(img).flatten()/255

In [131]:
x=model.predict(img_array.reshape(1,1024))
x=x.ravel()

1/1 [==============================] - 0s 153ms/step


In [132]:
nepali_dict={
    0: 'क', 1: 'ख', 2: 'ग', 3: 'घ', 4: 'ङ', 5: 'च', 6: 'छ', 7: 'ज', 8: 'झ', 9: 'ञ',
    10: 'ट', 11: 'ठ', 12: 'ड', 13: 'ढ', 14: 'ण', 15: 'त', 16: 'थ', 17: 'द', 18: 'ध', 19: 'न',
    20: 'प', 21: 'फ', 22: 'ब', 23: 'भ', 24: 'म', 25: 'य', 26: 'र', 27: 'ल', 28: 'व', 29: 'श',
    30: 'ष', 31: 'स', 32: 'ह', 33: 'क्ष', 34: 'त्र', 35: 'ज्ञ'
}


In [133]:
print("the enterd string is :",nepali_dict[np.argmax(x)])

the enterd string is : क
